In [53]:
import pandas as pd 

#Loading data
data = pd.read_csv("../data/example.csv")

race = data["race"]
religion = data["religion"]
country = data["country"]
cr_t = data["time_criminal"]
toxic_t = data["time_toxic"]
tests = []
tests.append(data["test_1"])
tests.append(data["test_2"])
tests.append(data["test_3"])
tests.append(data["test_4"])
tests.append(data["test_5"])
tests.append(data["test_6"])
tests.append(data["test_7"])

#Filtering out test according to the quality control
#The test with the golden question
golden_test = 3
#The tests that are shuffled version of the same models
same_model_1 = 0
same_model_2 = 5

num_tests = len(tests)
num_data = len(tests[0])

filtered_tests = []
for i in range(num_tests):
    filtered_tests.append([])
    
for i in range(num_data):
    if tests[golden_test][i] == 5 and tests[same_model_1][i] == tests[same_model_2][i]:
        for j in range(num_tests):
            filtered_tests[j].append(tests[j][i]) 
 
#print(filtered_tests)
#Calculating task times    
tests = filtered_tests
num_data = len(tests[0])

avg_cr_t = 0
for t in cr_t:
    avg_cr_t += t
avg_cr_t = avg_cr_t / len(cr_t)

avg_toxic_t = 0
for t in toxic_t:
    avg_toxic_t += t
avg_toxic_t = avg_toxic_t / len(toxic_t)

print("The average finish time of criminal justice task: " + str(avg_cr_t))
print("The average finish time of toxic comments task: " + str(avg_toxic_t))

#Calculating the number of don't know answers
counts = [0] * num_data
for i in range(num_tests):
    for j in range(num_data):
        if tests[i][j] == 3:
            counts[j] += 1

avg_dont_know = 0
for j in range(num_data):
    avg_dont_know += counts[j]
    
avg_dont_know = avg_dont_know / num_data

print("The average number of don't know answers: " + str(avg_dont_know))



The average finish time of criminal justice task: 575.3333333333334
The average finish time of toxic comments task: 596.6666666666666
The average number of don't know answers: 2.0
